The purpose of this notebook is to demonstrate the prototype gene mapper library put together to generalize the processes of

- Mapping gene symbols to gene identifiers
- Mapping gene identifiers across authorities (NCBI, ENSEMBL, etc.)
- Mapping genes in one species to ortholog genes in another species

for MapMyCells.

In addition to the `mmc_gene_mapper` codebase and its dependencies, this notebook depends on the `abc_atlas_access` tool, though only for downloading metadata to simulate mapping 10s of thousands of genes at once (as we would with real user data).

## Setup

In [1]:
import h5py
import json
import pathlib
import subprocess
import sqlite3
import time

import abc_atlas_access.abc_atlas_cache.abc_project_cache as abc_cache_library

import mmc_gene_mapper
import mmc_gene_mapper.utils.file_utils as file_utils
import mmc_gene_mapper.mapper.mapper as mapper

In [2]:
abc_cache = abc_cache_library.AbcProjectCache.from_cache_dir('../data/abc_cache')

In [3]:
data_dir = pathlib.Path(
    mmc_gene_mapper.__file__).parent.parent.parent / "data"
assert data_dir.is_dir()

In [4]:
db_path = data_dir/"gene_mapper_example.db"
assert db_path.parent.is_dir()

There are some files in
```
/allen/scratch/aibstemp/danielsf/gene_mapper_data
```
that are used to ingest data beyond what is available in the NCBI FTP server. Please copy those files to `../data/db_creation_data` in this respository (the cell below will fail if any of the files are not present)

In [5]:
data_file_spec=[
    {"type": "bkbit",
     "path": "../data/db_creation_data/mouse_ENSEMBL-10090-98.jsonld"},
    {"type": "bkbit",
     "path": "../data/db_creation_data/human_ENSEMBL-9606-101.jsonld"},
    {"type": "hmba_orthologs",
     "path": "../data/db_creation_data/all_gene_ids.csv",
     "name": "HMBA",
     "baseline_species": "human"}
]

error_msg = ""
for spec in data_file_spec:
    pth = pathlib.Path(spec['path'])
    if not pth.is_file():
        error_msg += f"{pth} is not a file\n"
if len(error_msg) > 0:
    raise RuntimeError(error_msg)

Below we instantiate the class that is used to actually do the mapping. The first time you run the cell, it will take ~ 15 minutes as it downloads ~ 4 GB of data from NCBI and ingests that data, alongside the local data from the cell above, to create an ~ 8 GB sqlite database that backs the mapping process.

As long as you leave `clobber=False` below, you will not have to recreate that database on subsequent uses.

**Note:** running this cell will also produce some warnings about genes that could not be ingested into the ortholog tables. These are due to genes in the ortholog table that have been deprecated by NCBI since the ortholog table (especially our by-hand HMBA ortholog table) was generated, meaning the gene mapper cannot connect them to species. We will have to think about how to handle versioning mismatches of this sort.

In [6]:
gene_mapper = mapper.MMCGeneMapper(
    db_path=db_path,
    local_dir=data_dir,
    data_file_spec=data_file_spec,
    clobber=False,
    force_download=False,
    suppress_download_stdout=True
)

=======CREATING DB FILE=======
=======CREATING TABLES=======
=======DOWNLOADING gene/DATA/gene_info.gz=======
=======DOWNLOADED gene/DATA/gene_info.gz in 9.25e+02 seconds=======
=======DOWNLOADING gene/DATA/gene2ensembl.gz=======
=======DOWNLOADED gene/DATA/gene2ensembl.gz in 6.89e+01 seconds=======
=======DOWNLOADING gene/DATA/gene_orthologs.gz=======
=======DOWNLOADED gene/DATA/gene_orthologs.gz in 8.11e+01 seconds=======
=======INGESTING GENE INFO=======
    chunk 0.00e+00, 5.00e+06
    chunk 5.00e+06, 1.00e+07
    chunk 1.00e+07, 1.50e+07
    chunk 1.50e+07, 2.00e+07
    chunk 2.00e+07, 2.50e+07
    chunk 2.50e+07, 3.00e+07
    chunk 3.00e+07, 3.50e+07
    chunk 3.50e+07, 4.00e+07
    chunk 4.00e+07, 4.50e+07
    chunk 4.50e+07, 5.00e+07
    chunk 5.00e+07, 5.50e+07
    chunk 5.50e+07, 6.00e+07
    chunk 6.00e+07, 6.03e+07
=======INGESTING gene_info TOOK 2.72e+00 minutes=======
=======INGESTING GENE TO ENSEMBL=======
=======INGESTING gene2ensembl TOOK 1.96e+00 minutes=======
======

/Users/scott.daniel/KnowledgeBase/garage/mmc_gene_mapper/src/mmc_gene_mapper/create_db/ortholog_ingestion.py:383: InvalidOrthologGeneWarning: The following genes had no species assigned to them and were not ingested
[3512700]
  warnings.warn(warning_msg, category=InvalidOrthologGeneWarning)


inserting 15620263 values with citation 0
=======INGESTING gene_orthologs TOOK 3.71e+00 minutes=======
=======DOWNLOADING pub/taxonomy/new_taxdump/new_taxdump.tar.gz=======
=======DOWNLOADED pub/taxonomy/new_taxdump/new_taxdump.tar.gz in 1.01e+02 seconds=======
=======DOWNLOADING pub/taxonomy/new_taxdump/new_taxdump.tar.gz.md5=======
=======DOWNLOADED pub/taxonomy/new_taxdump/new_taxdump.tar.gz.md5 in 3.15e-01 seconds=======
=======INGESTING ../data/db_creation_data/mouse_ENSEMBL-10090-98.jsonld=======
    INGESTING 55421 GENES
=======INGESTING ../data/db_creation_data/human_ENSEMBL-9606-101.jsonld=======
    INGESTING 60671 GENES
=======INGESTING ORTHOLOGS FROM all_gene_ids.csv=======
=======DELETING INDEX tmp_gene_to_species_idx======
=======CREATING INDEX tmp_gene_to_species_idx=======
=======DELETING INDEX tmp_gene_to_species_idx======
gene0 681185
gene1 681185
    GOT GRAPH
    group 1 decrement 2.600e+01 of 7.003e+05
    group 7 decrement 2.750e+02 of 7.003e+05
    group 69 decre

/Users/scott.daniel/KnowledgeBase/garage/mmc_gene_mapper/src/mmc_gene_mapper/create_db/ortholog_ingestion.py:383: InvalidOrthologGeneWarning: The following genes had no species assigned to them and were not ingested
[105466403, 105470582, 105496978, 108175750, 100343007, 105492737, 103246736, 105470573, 110089133, 103222958, 127492382, 127492411, 127488423, 119622548, 105468309, 127488420, 127492371, 105489432, 127488336, 119618156, 110085356, 112426108, 131272925, 103239149, 127488402, 105498501, 127482665, 105465210, 105465117, 103220181, 101419832, 127490343, 101429552, 110079788, 112422929, 110070386, 105488485, 119624761, 100008594, 100356693, 127483265, 127483264, 119618272, 110091021, 110085873, 110076977, 127490732, 110074157, 100229482, 127485497, 110081192, 110083144, 112422919, 105473341, 119621279, 123378922, 100345490, 100353604, 127487406, 127492160, 100354248, 100348221, 127487409, 105466118, 105473490, 119618136, 100341680, 110070272, 103218761, 119627299, 101413543, 10

inserting 699798 values with citation 3
=======ORTHOLOG INGESTION TOOK 1.61e+00 minutes=======
=======DELETING INDEX gene_symbol_idx======
=======CREATING INDEX gene_symbol_idx=======
=======DELETING INDEX gene_id_idx======
=======CREATING INDEX gene_id_idx=======
=======DELETING INDEX gene_equivalence_idx======
=======CREATING INDEX gene_equivalence_idx=======
=======DELETING INDEX gene_ortholog_gene_idx======
=======CREATING INDEX gene_ortholog_gene_idx=======
=======DELETING INDEX gene_ortholog_ortholog_idx======
=======CREATING INDEX gene_ortholog_ortholog_idx=======
=======CREATING BIBLIOGRAPHY TABLE=======
=======DELETING INDEX species_bibliography_idx======
=======DELETING INDEX species_bibliography_idx======
=======CREATING INDEX species_bibliography_idx=======
=======COPYING TMP FILE TO /Users/scott.daniel/KnowledgeBase/garage/mmc_gene_mapper/data/gene_mapper_example.db=======
=======DB CREATION TOOK 3.30e+01 MINUTES=======


In [7]:
gene_mapper.db_path

PosixPath('/Users/scott.daniel/KnowledgeBase/garage/mmc_gene_mapper/data/gene_mapper_example.db')

## Contents of the database backing the mapper

The database backing the `gene_mapper` contains the following tables

### Metadata tables

#### authority
The `authority` table just tracks the gene identifying authorities the database knows about (i.e. ENSEMBL and NCBI). it consists of the columns

- id -- an integer used for internal indexing
- name -- the human-readable name of the authority

#### citation
The `citation` table tracks all of the datasets used to justify the various mappings (symbol-to-identifier, ENSEMBL-to-NCBI, orthologs, etc.). It consists of the columns

- id -- an integer used for internal indexing
- name -- a short, human-readable name for the citation (this will be used whenever you have to specify a citation when calling the `gene_mapper`'s methods
- metadata -- a JSON serialized dict containing whatever information is necessary to recreate or specify the data backing this citation. This is a very free-form column as different entries will be specified in different ways (see below)

We can list the available authorities like so

In [8]:
authority_list = gene_mapper.get_all_authorities()
print(authority_list)

['NCBI', 'ENSEMBL']


Similarly, we can list the available citations.

In [9]:
citation_list = gene_mapper.get_all_citations()
print([c['name'] for c in citation_list])

['NCBI', 'ENSEMBL-10090-98', 'ENSEMBL-9606-101', 'HMBA']


As mentioned above, different citations contain different metadata. For instance, the `NCBI` citation (referring to the data downloaded from the NCBI FTP server) contains a list of the files downloaded their hashes, and the date they were downloaded on (NCBI updates its FTP server daily; it's not clear that they keep readily available version information around, though we can certainly investigate that). This is what the `NCBI` citation metadata looks like:

In [10]:
for citation in citation_list:
    if citation['name'] == 'NCBI':
        print(json.dumps(citation, indent=2))

{
  "name": "NCBI",
  "metadata": {
    "gene_info.gz": {
      "host": "ftp.ncbi.nlm.nih.gov",
      "src_path": "gene/DATA/gene_info.gz",
      "hash": "md5:1a9ff7008f26126ec4f6cee6fca4d33b",
      "downloaded_on": "2025-04-18-15-24-22"
    },
    "gene2ensembl.gz": {
      "host": "ftp.ncbi.nlm.nih.gov",
      "src_path": "gene/DATA/gene2ensembl.gz",
      "hash": "md5:46faf9bae8f5eaa5d8c50f5b735c86af",
      "downloaded_on": "2025-04-18-15-25-32"
    },
    "gene_orthologs.gz": {
      "host": "ftp.ncbi.nlm.nih.gov",
      "src_path": "gene/DATA/gene_orthologs.gz",
      "hash": "md5:441883dd9d941c89e9625ad26cedb47b",
      "downloaded_on": "2025-04-18-15-26-53"
    }
  }
}


The citations based on BICAN bkbit files, contain all of the GenomeAnnotation and GenomeAssembly information in those files.

In [11]:
for citation in citation_list:
    if citation['name'] == 'ENSEMBL-10090-98':
        print(json.dumps(citation, indent=2))

{
  "name": "ENSEMBL-10090-98",
  "metadata": {
    "biolink:OrganismTaxon": {
      "id": "NCBITaxon:10090",
      "iri": "http://purl.obolibrary.org/obo/NCBITaxon_10090",
      "category": [
        "biolink:OrganismTaxon"
      ],
      "name": "house mouse",
      "full_name": "Mus musculus"
    },
    "bican:GenomeAssembly": {
      "id": "NCBIAssembly:GCF_000001635.26",
      "category": [
        "bican:GenomeAssembly"
      ],
      "name": "GRCm38",
      "in_taxon": [
        "NCBITaxon:10090"
      ],
      "in_taxon_label": "Mus musculus",
      "version": "26"
    },
    "bican:GenomeAnnotation": {
      "id": "bican:annotation-ENSEMBL-10090-98",
      "category": [
        "bican:GenomeAnnotation"
      ],
      "description": "ENSEMBL Mus musculus Annotation Release 98",
      "in_taxon": [
        "NCBITaxon:10090"
      ],
      "in_taxon_label": "Mus musculus",
      "version": "98",
      "digest": [
        "urn:uuid:58618e83-7b4c-46ed-b300-9df8abf1f5d0"
      ],
  

The `HMBA` citation is an ingest of a lookup table created by our science team. As such, it has the least detailed information.

In [12]:
for citation in citation_list:
    if citation['name'] == 'HMBA':
        print(json.dumps(citation, indent=2))

{
  "name": "HMBA",
  "metadata": {
    "file": "../data/db_creation_data/all_gene_ids.csv",
    "hash": "md5:3a038d261dfde93c95c0c1d9ede39a24"
  }
}


### Data tables

The data tables in the database are

#### NCBI_species
The `NCBI_species` table just ingests the NCBI organism taxonomy and records
- id -- the NCBI taxon ID of the species
- name -- the human readable name of the species. **Note:** we record all possible names ("common", "scientific", etc.) of the species in different rows. That way, users can give us whatever name they have for a species and, if possible, we can return an `id` for cross-referencing with other tables.

This is meant to be hidden from the user. The user specifies the species when performing a mapping and the gene_mapper makes the necessary database call. Below, we will make the call "by hand", so you can see what is in the database.

In [13]:
with sqlite3.connect(gene_mapper.db_path) as conn:
    cursor = conn.cursor()
    for name in ('house mouse', 'Mus musculus', 'mouse', 'human', 'Homo sapiens'):
        result = cursor.execute(
            """
            SELECT id FROM NCBI_species WHERE name=?
            """,
            (name,)
        ).fetchall()
        print(name,result)

house mouse [(10090,)]
Mus musculus [(10090,)]
mouse [(10088,), (10090,)]
human [(9606,)]
Homo sapiens [(9606,)]


**Note:** in the case of "mouse", there is more than one matching taxon. Calling a mapping with `species="mouse"` will fail.

In [14]:
mouse_symbols = ["Xkr4", "Npbwr1", "not_a_symbol", "Rrs1"]
gene_mapper.identifiers_from_symbols(
    gene_symbol_list=mouse_symbols,
    species_name="mouse",
    authority_name="ENSEMBL"
)

ValueError: 2 species match name mouse
[(10088,), (10090,)]

A function is provided for listing all of the species the database knows about.

**Note:** This is not a list of all of the species for which the database has gene information. It is merely a list of the species that can be associated with a taxon ID in the database (for cross-referencing with the tables that contain gene information).

In [15]:
species_list = gene_mapper.get_all_species()
print(species_list[2000000:2000005])

['Lathyrus setifolius', 'Lathyrus setifolius L., 1753', 'Lathyrus setiger', 'Lathyrus setiger Phil., 1864', 'Lathyrus sp. DRC-2004']


In [16]:
'Homo sapiens' in species_list

True

In [17]:
len(species_list)

4069125

#### gene
The `gene` table contains all of the information about individual genes ingested into the database. Columns are
- authority -- an integer for cross-referencing the `authority` table
- citation -- an integer for cross-referencing the `citation` table
- species_taxon -- an integer for cross-refereneing the `NCBI_species` table
- id -- an integer for internal indexing
- identifier -- a string. The full unique identifier (e.g. ENSMUS0000G12345) of the gene
- symbol -- a string. The human-readable name of the gene (**Note:** the BICAN bkbit files contain `symbol` and `name` entries for genes. If these differ, both are ingested as a separate row in the `gene` table with `name` being recorded in `symbol` as appropriate).

#### gene_equivalence
The `gene_equivalence` table records mappings between authorities (ENSEMBL and NCBI). Its columns are
- species_taxon -- an integer for cross-referencing with the `NCBI_species` table (in what species are we looking for equivalences?)
- citation -- an integer for cross-referencing with the `citation` table (who said these genes were equivalent?)
- authority0 -- an integer for cross-referencing with the `authority` table
- gene0 -- an integer; the ID of the gene in `authority0`
- authority1 -- an integer for cross-referncing with the `authority` table
- gene1 -- an integer; the ID of the gene in `authority1` that is equivalent to `gene0` according to `citation`.

**Note:** we record equivalences symmetrically, so that every `(gene0, gene1)` pair is also recorded as `(gene1, gene0)` so that users do not have to worry about which index (`gene0` or `gene1`) their input data is compared to.

#### gene_ortholog
The `gene_ortholog` table records cross-species ortholog relationships. Its columns are
- authority -- an integer for cross-referencing the `authority` table (are we looking for NCBI orthologs or ENSEMBL orthologs?)
- citation -- an integer for cross-referencing the `citation` table (who said these genes were orthologs?)
- species -- an integer for cross-referencing the `NCBI_species` table
- gene -- an integer, the ID of the gene in `species` whose orthologs we are looking for
- ortholog_group -- an integer; genes with the same value of `ortholog_group` are orthologs of each other


## Actually mapping data

The `gene_mapper` provides funtions to perform the various mappings we support, namely
- gene symbol to gene identifier within one authority
- identifier to identifier across authorities
- identifier to identifier across species within one authority

The functions return a dict containing all of the relevant mappings, including 1:N mappings (it is left to the user to decide what to do in those cases) along with the metadata necessary to understand the provenance of the mapping.

### Mapping symbols to identifiers

For instance, here is a function call mapping gene symbols to gene identifiers in ENSEMBL

In [18]:
mouse_symbols = ["Xkr4", "Npbwr1", "not_a_symbol", "Rrs1"]
mouse_ens_mapping = gene_mapper.identifiers_from_symbols_mapping(
    gene_symbol_list=mouse_symbols,
    species_name="Mus musculus",
    authority_name="ENSEMBL"
)
print(json.dumps(mouse_ens_mapping, indent=2))

{
  "metadata": {
    "authority": {
      "name": "ENSEMBL",
      "idx": 1
    },
    "citation": {
      "name": "ENSEMBL-10090-98",
      "metadata": {
        "biolink:OrganismTaxon": {
          "id": "NCBITaxon:10090",
          "iri": "http://purl.obolibrary.org/obo/NCBITaxon_10090",
          "category": [
            "biolink:OrganismTaxon"
          ],
          "name": "house mouse",
          "full_name": "Mus musculus"
        },
        "bican:GenomeAssembly": {
          "id": "NCBIAssembly:GCF_000001635.26",
          "category": [
            "bican:GenomeAssembly"
          ],
          "name": "GRCm38",
          "in_taxon": [
            "NCBITaxon:10090"
          ],
          "in_taxon_label": "Mus musculus",
          "version": "26"
        },
        "bican:GenomeAnnotation": {
          "id": "bican:annotation-ENSEMBL-10090-98",
          "category": [
            "bican:GenomeAnnotation"
          ],
          "description": "ENSEMBL Mus musculus Annotation 

For ease of reading, here is the output broken down into metadata

In [19]:
print(json.dumps(mouse_ens_mapping['metadata'], indent=2))

{
  "authority": {
    "name": "ENSEMBL",
    "idx": 1
  },
  "citation": {
    "name": "ENSEMBL-10090-98",
    "metadata": {
      "biolink:OrganismTaxon": {
        "id": "NCBITaxon:10090",
        "iri": "http://purl.obolibrary.org/obo/NCBITaxon_10090",
        "category": [
          "biolink:OrganismTaxon"
        ],
        "name": "house mouse",
        "full_name": "Mus musculus"
      },
      "bican:GenomeAssembly": {
        "id": "NCBIAssembly:GCF_000001635.26",
        "category": [
          "bican:GenomeAssembly"
        ],
        "name": "GRCm38",
        "in_taxon": [
          "NCBITaxon:10090"
        ],
        "in_taxon_label": "Mus musculus",
        "version": "26"
      },
      "bican:GenomeAnnotation": {
        "id": "bican:annotation-ENSEMBL-10090-98",
        "category": [
          "bican:GenomeAnnotation"
        ],
        "description": "ENSEMBL Mus musculus Annotation Release 98",
        "in_taxon": [
          "NCBITaxon:10090"
        ],
        "i

and actual mapping

In [20]:
print(json.dumps(mouse_ens_mapping['mapping'], indent=2))

{
  "Xkr4": [
    "ENSMUSG00000051951"
  ],
  "Npbwr1": [
    "ENSMUSG00000033774"
  ],
  "not_a_symbol": [],
  "Rrs1": [
    "ENSMUSG00000061024"
  ]
}


Here is a similar call mapping symbols to identifiers in NCBI

In [21]:
mouse_ncbi_mapping = gene_mapper.identifiers_from_symbols_mapping(
    gene_symbol_list=mouse_symbols,
    species_name="Mus musculus",
    authority_name="NCBI"
)
print(json.dumps(mouse_ncbi_mapping, indent=2))

{
  "metadata": {
    "authority": {
      "name": "NCBI",
      "idx": 0
    },
    "citation": {
      "name": "NCBI",
      "metadata": {
        "gene_info.gz": {
          "host": "ftp.ncbi.nlm.nih.gov",
          "src_path": "gene/DATA/gene_info.gz",
          "hash": "md5:1a9ff7008f26126ec4f6cee6fca4d33b",
          "downloaded_on": "2025-04-18-15-24-22"
        },
        "gene2ensembl.gz": {
          "host": "ftp.ncbi.nlm.nih.gov",
          "src_path": "gene/DATA/gene2ensembl.gz",
          "hash": "md5:46faf9bae8f5eaa5d8c50f5b735c86af",
          "downloaded_on": "2025-04-18-15-25-32"
        },
        "gene_orthologs.gz": {
          "host": "ftp.ncbi.nlm.nih.gov",
          "src_path": "gene/DATA/gene_orthologs.gz",
          "hash": "md5:441883dd9d941c89e9625ad26cedb47b",
          "downloaded_on": "2025-04-18-15-26-53"
        }
      },
      "idx": 0
    }
  },
  "mapping": {
    "Xkr4": [
      "NCBIGene:497097"
    ],
    "Npbwr1": [
      "NCBIGene:226304"
    ],


### Mapping from ENSEMBL to NCBI

Below is a function call mapping ENSEMBL IDs to NCBI IDs using the gene equivalences downloaded from the NCBI FTP server. Note that all of the mappings map one ENSEMBL ID to two NCBI IDs.

In [22]:
ens_to_ncbi = gene_mapper.equivalent_genes_mapping(
    input_authority='ENSEMBL',
    output_authority='NCBI',
    gene_list=["ENSMUSG00000030337", "ENSMUSG00000037747", "nope", "ENSMUSG00000021983"],
    species_name="Mus musculus",
    citation_name="NCBI"
)
print(json.dumps(ens_to_ncbi, indent=2))

{
  "metadata": {
    "key_authority": "ENSEMBL",
    "value_authority": "NCBI",
    "citation": {
      "name": "NCBI",
      "idx": 0,
      "metadata": {
        "gene_info.gz": {
          "host": "ftp.ncbi.nlm.nih.gov",
          "src_path": "gene/DATA/gene_info.gz",
          "hash": "md5:1a9ff7008f26126ec4f6cee6fca4d33b",
          "downloaded_on": "2025-04-18-15-24-22"
        },
        "gene2ensembl.gz": {
          "host": "ftp.ncbi.nlm.nih.gov",
          "src_path": "gene/DATA/gene2ensembl.gz",
          "hash": "md5:46faf9bae8f5eaa5d8c50f5b735c86af",
          "downloaded_on": "2025-04-18-15-25-32"
        },
        "gene_orthologs.gz": {
          "host": "ftp.ncbi.nlm.nih.gov",
          "src_path": "gene/DATA/gene_orthologs.gz",
          "hash": "md5:441883dd9d941c89e9625ad26cedb47b",
          "downloaded_on": "2025-04-18-15-26-53"
        }
      }
    }
  },
  "mapping": {
    "ENSMUSG00000021983": [
      "NCBIGene:50769",
      "NCBIGene:108168164"
    ],
    "E

Here is a call mapping from NCBI to ENSEMBL

In [23]:
ncbi_to_ens = gene_mapper.equivalent_genes_mapping(
    input_authority='NCBI',
    output_authority='ENSEMBL',
    gene_list=["NCBIGene:67269", "NCBIGene:54200", "nope", "NCBIGene:70911"],
    species_name="Mus musculus",
    citation_name="NCBI"
)
print(json.dumps(ncbi_to_ens, indent=2))

{
  "metadata": {
    "key_authority": "NCBI",
    "value_authority": "ENSEMBL",
    "citation": {
      "name": "NCBI",
      "idx": 0,
      "metadata": {
        "gene_info.gz": {
          "host": "ftp.ncbi.nlm.nih.gov",
          "src_path": "gene/DATA/gene_info.gz",
          "hash": "md5:1a9ff7008f26126ec4f6cee6fca4d33b",
          "downloaded_on": "2025-04-18-15-24-22"
        },
        "gene2ensembl.gz": {
          "host": "ftp.ncbi.nlm.nih.gov",
          "src_path": "gene/DATA/gene2ensembl.gz",
          "hash": "md5:46faf9bae8f5eaa5d8c50f5b735c86af",
          "downloaded_on": "2025-04-18-15-25-32"
        },
        "gene_orthologs.gz": {
          "host": "ftp.ncbi.nlm.nih.gov",
          "src_path": "gene/DATA/gene_orthologs.gz",
          "hash": "md5:441883dd9d941c89e9625ad26cedb47b",
          "downloaded_on": "2025-04-18-15-26-53"
        }
      }
    }
  },
  "mapping": {
    "NCBIGene:54200": [
      "ENSMUSG00000003271"
    ],
    "NCBIGene:67269": [
      "ENS

To simulate "production scale," we will now download all of the genes in the Yao et al. 2023 10X gene panel and try mappping them from ENSEMBL to NCBI to see how long it takes to process that many genes.

In [24]:
wmb_gene_df = abc_cache.get_metadata_dataframe(
    directory='WMB-10X',
    file_name='gene'
)
wmb_genes = wmb_gene_df.gene_identifier.values
t0 = time.time()
wmb_ens_to_ncbi = gene_mapper.equivalent_genes_mapping(
    input_authority='ENSEMBL',
    output_authority='NCBI',
    gene_list=wmb_genes,
    species_name='Mus musculus',
    citation_name='NCBI'
)
dur = time.time()-t0
print(f'mapping {len(wmb_genes)} genes took {dur:.2e} seconds')

n_mapped = 0
n_one_to_one = 0
for k in wmb_genes:
    if len(wmb_ens_to_ncbi['mapping'][k]) > 0:
        n_mapped += 1
    if len(wmb_ens_to_ncbi['mapping'][k]) == 1:
        n_one_to_one += 1
print(f'{n_mapped} genes had NCBI matches')
print(f'{n_one_to_one} genes had exactly one NCBI match')

gene.csv: 100%|███████████████████████████| 2.30M/2.30M [00:00<00:00, 4.71MMB/s]


mapping 32285 genes took 1.88e+01 seconds
24656 genes had NCBI matches
24527 genes had exactly one NCBI match


Now we will download the marker genes used by MapMyCells for the Whole Mouse Brain taxonomy and try mapping them from ENSEMBL to NCBI.

In [25]:
src_path = "https://allen-brain-cell-atlas.s3-us-west-2.amazonaws.com/mapmycells/WMB-10X/20240831/mouse_markers_230821.json"
wmb_marker_path = pathlib.Path('../data/mouse_markers_230821.json')
if not wmb_marker_path.is_file():
    print(f"downloading {wmb_marker_path}")
    process = subprocess.Popen(
        args=[
           "wget",
            src_path,
            "-O",
            str(wmb_marker_path),
        ],
        stderr=subprocess.DEVNULL
    )
    process.wait()

downloading ../data/mouse_markers_230821.json


In [26]:
marker_genes = set()
with open(wmb_marker_path, 'rb') as src:
    marker_lookup = json.load(src)
for key in marker_lookup:
    if key in ('log', 'metadata'):
        continue
    marker_genes = marker_genes.union(set(marker_lookup[key]))
marker_genes = sorted(marker_genes)

Let's now map our mouse marker genes from ENSEMBL to NCBI

In [27]:
print(f'{len(marker_genes)} marker genes')
t0 = time.time()
marker_ens_to_ncbi = gene_mapper.equivalent_genes_mapping(
    input_authority='ENSEMBL',
    output_authority='NCBI',
    gene_list=marker_genes,
    species_name='Mus musculus',
    citation_name='NCBI'
)
dur = time.time()-t0
print(f'mappiing {len(marker_genes)} genes took {dur:.2e} seconds')

n_mapped = 0
n_one_to_one = 0
for k in marker_genes:
    if len(marker_ens_to_ncbi['mapping'][k]) > 0:
        n_mapped += 1
    if len(marker_ens_to_ncbi['mapping'][k]) == 1:
        n_one_to_one += 1
print(f'{n_mapped} genes had NCBI matches')
print(f'{n_one_to_one} genes had exactly one NCBI match')

6558 marker genes
mappiing 6558 genes took 1.35e+00 seconds
6525 genes had NCBI matches
6493 genes had exactly one NCBI match


### Ortholog mapping

To test ortholog mapping, let's take the genes from Yao et al. 2023 that only mapped to one NCBI gene and map them to human orthologs.

In [28]:
wmb_ncbi_genes = [
    wmb_ens_to_ncbi['mapping'][ens][0]
    for ens in wmb_genes
    if len(wmb_ens_to_ncbi['mapping'][ens]) == 1
]

First let's just map 5 genes so that we can display the output and see what it looks like

In [29]:
test_orthologs = gene_mapper.ortholog_genes_mapping(
    authority='NCBI',
    src_species_name='Mus musculus',
    dst_species_name='human',
    gene_list=wmb_ncbi_genes[:5] + ['not_an_actual_gene'],
    citation_name='NCBI'
)
print(json.dumps(test_orthologs, indent=2))

{
  "metadata": {
    "authority": "NCBI",
    "citation": {
      "gene_info.gz": {
        "host": "ftp.ncbi.nlm.nih.gov",
        "src_path": "gene/DATA/gene_info.gz",
        "hash": "md5:1a9ff7008f26126ec4f6cee6fca4d33b",
        "downloaded_on": "2025-04-18-15-24-22"
      },
      "gene2ensembl.gz": {
        "host": "ftp.ncbi.nlm.nih.gov",
        "src_path": "gene/DATA/gene2ensembl.gz",
        "hash": "md5:46faf9bae8f5eaa5d8c50f5b735c86af",
        "downloaded_on": "2025-04-18-15-25-32"
      },
      "gene_orthologs.gz": {
        "host": "ftp.ncbi.nlm.nih.gov",
        "src_path": "gene/DATA/gene_orthologs.gz",
        "hash": "md5:441883dd9d941c89e9625ad26cedb47b",
        "downloaded_on": "2025-04-18-15-26-53"
      }
    },
    "src_species_taxon": 10090,
    "dst_species_taxon": 9606
  },
  "mapping": {
    "NCBIGene:19888": [
      "NCBIGene:6101"
    ],
    "NCBIGene:20671": [
      "NCBIGene:64321"
    ],
    "NCBIGene:27395": [
      "NCBIGene:29088"
    ],
    "NCB

Now let's map all 24,000 genes and see how long that takes

In [30]:
t0 = time.time()
wmb_to_whb_orthologs = gene_mapper.ortholog_genes_mapping(
    authority='NCBI',
    src_species_name='Mus musculus',
    dst_species_name='human',
    gene_list=wmb_ncbi_genes,
    citation_name='NCBI'
)
dur = time.time()-t0
print(f'mapping {len(wmb_ncbi_genes)} orthologs took {dur:.2e} seconds')
n_mapped = 0
n_one_to_one = 0
for k in wmb_ncbi_genes:
    if len(wmb_to_whb_orthologs['mapping'][k]) > 0:
        n_mapped += 1
    if len(wmb_to_whb_orthologs['mapping'][k]) == 1:
        n_one_to_one += 1
print(f'{n_mapped} genes had orthologs')
print(f'{n_one_to_one} genes had exactly one ortholog')

mapping 24527 orthologs took 9.33e+00 seconds
16508 genes had orthologs
16508 genes had exactly one ortholog


To map orthologs using the "by-hand" look up table created by our science team, change the kwarg `citation='HMBA'`

In [31]:
t0 = time.time()
wmb_to_whb_orthologs_hmba = gene_mapper.ortholog_genes_mapping(
    authority='NCBI',
    src_species_name='Mus musculus',
    dst_species_name='human',
    gene_list=wmb_ncbi_genes,
    citation_name='HMBA'
)
dur = time.time()-t0
print(f'mapping {len(wmb_ncbi_genes)} orthologs took {dur:.2e} seconds')
n_mapped = 0
n_one_to_one = 0
for k in wmb_ncbi_genes:
    if len(wmb_to_whb_orthologs_hmba['mapping'][k]) > 0:
        n_mapped += 1
    if len(wmb_to_whb_orthologs_hmba['mapping'][k]) == 1:
        n_one_to_one += 1
print(f'{n_mapped} genes had orthologs')
print(f'{n_one_to_one} genes had exactly one ortholog')

mapping 24527 orthologs took 9.23e+00 seconds
16450 genes had orthologs
16450 genes had exactly one ortholog


#### Map from mouse to naked mole rat

Now, inspired by a recent community forum post, let's take our mouse marker genes, convert them into NCBI identifiers, and finally convert those in to naked mole rat orthologs.

In [32]:
t0 = time.time()

marker_ens_to_ncbi = gene_mapper.equivalent_genes_mapping(
    input_authority='ENSEMBL',
    output_authority='NCBI',
    gene_list=marker_genes,
    species_name='Mus musculus',
    citation_name='NCBI'
)

marker_as_ncbi = [
    marker_ens_to_ncbi['mapping'][m][0]
    for m in marker_genes
    if m in marker_ens_to_ncbi['mapping'] and len(marker_ens_to_ncbi['mapping'][m]) == 1
]

naked_mole_rat = gene_mapper.ortholog_genes_mapping(
    authority='NCBI',
    src_species_name='Mus musculus',
    dst_species_name='naked mole rat',
    gene_list=marker_as_ncbi,
    citation_name='NCBI'
)

dur = time.time()-t0
print(f"mapping took {dur:.2e} seconds")

mapping took 2.61e+00 seconds


In [33]:
only_one = 0
for k in naked_mole_rat['mapping']:
    if len(naked_mole_rat['mapping'][k]) == 1:
        only_one += 1
print(f'{only_one} markers of {len(marker_genes)} had a unique naked mole rat ortholog')

5619 markers of 6558 had a unique naked mole rat ortholog


Here is the metadata for the naked mole rat ortholog mapping

In [34]:
naked_mole_rat['metadata']

{'authority': 'NCBI',
 'citation': {'gene_info.gz': {'host': 'ftp.ncbi.nlm.nih.gov',
   'src_path': 'gene/DATA/gene_info.gz',
   'hash': 'md5:1a9ff7008f26126ec4f6cee6fca4d33b',
   'downloaded_on': '2025-04-18-15-24-22'},
  'gene2ensembl.gz': {'host': 'ftp.ncbi.nlm.nih.gov',
   'src_path': 'gene/DATA/gene2ensembl.gz',
   'hash': 'md5:46faf9bae8f5eaa5d8c50f5b735c86af',
   'downloaded_on': '2025-04-18-15-25-32'},
  'gene_orthologs.gz': {'host': 'ftp.ncbi.nlm.nih.gov',
   'src_path': 'gene/DATA/gene_orthologs.gz',
   'hash': 'md5:441883dd9d941c89e9625ad26cedb47b',
   'downloaded_on': '2025-04-18-15-26-53'}},
 'src_species_taxon': 10090,
 'dst_species_taxon': 10181}